In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

import mar
import preprocessing

In [2]:
data_dir = os.path.join(os.path.dirname(os.getcwd()), "data")
pat_num = 9

In [ ]:
mar = preprocessing.load_mar_data(data_dir, pat_num)
sickbay = preprocessing.load_sickbay_data(data_dir, pat_num)
accel = preprocessing.load_accel_data(data_dir, pat_num)
accel = preprocessing.process_accel_data(accel)
ecg = preprocessing.load_ecg_data(data_dir, pat_num)
sbs = preprocessing.load_retro_data(data_dir, pat_num, ignore_default=True)

In [ ]:
accel = preprocessing.load_accel_data(data_dir, pat_num)
accel = preprocessing.process_accel_data(accel)

print(accel)

100%|██████████| 95590/95590 [00:57<00:00, 1669.32it/s]

                     time         x         y         z         a
0     2024-01-15 12:20:00 -0.089844 -0.030859 -1.010156  1.014613
1     2024-01-15 12:20:02 -0.087891 -0.033984 -1.005469  1.009875
2     2024-01-15 12:20:04 -0.087891 -0.035156 -1.005859  1.010304
3     2024-01-15 12:20:06 -0.090234 -0.033203 -1.006250  1.010833
4     2024-01-15 12:20:08 -0.088672 -0.037109 -1.006641  1.011220
...                   ...       ...       ...       ...       ...
95585 2024-01-17 17:26:40 -0.623047  0.375000 -0.633594  0.964496
95586 2024-01-17 17:26:42 -0.994531 -0.368359 -0.881641  1.379156
95587 2024-01-17 17:26:44 -0.502344  0.478125 -0.751953  1.022930
95588 2024-01-17 17:26:46 -0.386328  0.322656 -0.893750  1.025741
95589 2024-01-17 17:26:48  0.010937 -0.062891 -0.993359  0.995408

[95590 rows x 5 columns]
